# Домашнее задание №6. Ансамбли: случайный лес, градиентный бустинг.

## Ф.И.О: Бедрин Павел Александрович

### Описание.

Домашнее задание состоит из 2-х частей:
  - реализация модулей:
    - bagger
    - booster
    - sampler
-------------------
 * Реализации можно частично проверить через юнит-тесты (запускаются командой ``pytest tests``).
-------------------
  - экспериментальная часть

На проверку требуется отправить zip архив, который будет содержать следующие файлы:
  - модуль ``ensemble`` с реализованными модулями(bagger, booster, sampler)
  - заполненный блокнот (эксперименты) в формате .ipynb 
  - заполненный блокнот в формате .html (File -> Save and Export Notebook As -> HTML -> ...)

## 1.1 Сэмплирование случайных объектов и признаков (2 points)

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

`BaseSampler` класс, который будет упрощать семплирование различных подмассивов данных. 
Реализуйте метод:
  - `sample_indices`, который по числу сущностей `n_objects` возращает случайную подвыборку индексов.

Используйте атрибуты:
  - `self.random_state`, чтобы результаты семпплирования воспроиводились
  - `self.bootstrap`, если нужно выбрать случайную подвыборку с возвращением.

У класса `ObjectSampler` реализован метод:
  - `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них.

В классе `FeaturesSampler` реализован метод:
- `sample`, который возвращает случайную подвыборку признаков для объектов.

## 1.2. Бэггинг (5 points)

Суть бэггинга заключается в обучении нескольких "слабых" базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов.

В данном задании реализация **случайного леса будет бэггингом над решающими деревьями**. 

Реализация случайного леса представлена в классе `RandomForestClassifier`.

## 1.3. Градиентный бустинг (7 points)

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. 

Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. У каждого следующего алгоритма вычисляется ___вес___ $\omega$, с которым он входит в ансамбль. 

Также есть параметр скорости обучения (___learning rate___ - $lr$), который не позволяет алгоритму переобучитсья. 

Вес $\omega$ можно находить, используя одномерную оптимизацию. 

Рассмотрим процедуру обучения по шагам (будем рассматривать случай бинарной классификации c метками классов {0,1}, чтобы не усложнять жизнь):
1. Настройка базового алгоритма $b_0$ (в данном случае это решающее дерево).
    
    $$\text{Алгоритм настраиваются на $y$ с помощью функции MSE.}$$
    
2. Будем обозначать текущий небазовый алгоритм - $a$:
    
    $$a_i(x) = \sum_{j=0}^i \omega_j b_j(x) $$
    
3. Настройка базового алгоритма $b_i$ (обычно это регрессионное дерево):
    
    $$b_i = \arg \min_b \sum_{j=1}^l (b(x_j) + \nabla L(a_{i-1}(x_j), y))^2,$$
    т.е. выход очередного базового алгоритма подстраивается под антиградиент функции потерь
    
4. Настройка веса базового алгоритма $\omega_i$:
    
    $$\omega_i = \min_{\omega > 0} \sum_{j=1}^l L(a_{i-1} + \omega b_i(x_j), y) $$
    
Для задачи классфикации будем использовать логистическую функцию потерь. Немного упростим ее:

$$L = -y\log\sigma(a) - (1-y)\log(1 - \sigma(a)) = -\log(1 - \sigma(a)) - y \log \frac{\sigma(a)}{1 - \sigma(a)},$$
где $\sigma$ - функция сигмоиды. 

Ответ после очередного базового алгоритма надо прогонять через сигмоиду, т.к. не гарантируется, что ответы будут лежать на [0,1] - в этом особенность базового алгоритма (который является регрессионным).

Преобразуем:
$$\log (1 - \sigma(a)) = \log \frac{1}{1 + \exp(a)} = -\log(1 + \exp(a)) $$

$$\log (\frac{\sigma(a)}{1 - \sigma(a)}) = \log(\exp(a)) = a $$
 
Таким образом:

$$L = -ya + \log(1 + \exp(a))$$

Тогда будем вычислять градиент как:
 
$$\nabla L = - y + \sigma(a)$$

В классе `Booster` реализуйте методы:
* `_fit_first_estimator` – построение первой модели (первого приближения данных);
* `fit` – обучение алгоритма градиентного бустинга (обучение первой и последующих базовых моделей);
* `predict` – получение предсказаний алгоритма градиентного бустинга.

В классе `GradientBoostingClassifier` реализуйте методы:
* `_fit_base_estimator` - обучение очередной базовой модели;
* `_gradient` - расчет градиента функции ошибки;
* `_loss` - расчет функции ошибки (для одномерно оптимизации).

## 2. Эксперименты (6 points)

Скачайте датасейт для экспериментов: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Колонка с ответами - RainTommorow.

In [ ]:
%load_ext autoreload
%autoreload 2

In [136]:
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from ensemble import RandomForestClassifier, GradientBoostingClassifier

In [183]:
data = pd.read_csv('weatherAUS.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Выделите признаки год/месяц/день:

In [184]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day

Посмотрим какие года есть в выборке:

In [185]:
data['year'].value_counts()

year
2016    17934
2014    17885
2015    17885
2009    16789
2010    16782
2013    16415
2012    15409
2011    15407
2017     8623
2008     2270
2007       61
Name: count, dtype: int64

Разделите выборку на три части (train, val и test) по временному принципу:
    
* train - 2007-2014
* val - 2015
* test - 2016-2017

In [186]:
indexes = {
    'train': (data['year'] >= 2007) & (data['year'] <= 2014),
    'val': data['year'] == 2015,
    'test': (data['year'] >= 2016) & (data['year'] <= 2017),
}

Здесь вы можете делать всевозможные преобразования признаков. 

Для того, чтобы получить качество, необходимое для преодоления бейзлайна, вам достаточно закодировать все категориальные признаки с помощью `LabelEncoder`, а также разумно обработать пропущенные значения.

In [187]:
data.drop(['Date'], axis=1, inplace=True)

In [188]:
data.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
year               int32
month              int32
day                int32
dtype: object

Удалим признаки, в которых много пропусков, возьмём порог >25% пропусков

In [189]:
missing_values_ratio = data.isnull().sum().sort_values(ascending=False) / len(data)
print("Признак - Доля пропусков\n", missing_values_ratio)

columns_to_keep = missing_values_ratio[missing_values_ratio <= 0.25].index
print(f"Удалили признаки: {set(data.columns) - set(columns_to_keep)}")
data = data[columns_to_keep]

Признак - Доля пропусков
 Sunshine         0.480098
Evaporation      0.431665
Cloud3pm         0.408071
Cloud9am         0.384216
Pressure9am      0.103568
Pressure3pm      0.103314
WindDir9am       0.072639
WindGustDir      0.070989
WindGustSpeed    0.070555
Humidity3pm      0.030984
WindDir3pm       0.029066
Temp3pm          0.024811
RainTomorrow     0.022460
RainToday        0.022419
Rainfall         0.022419
WindSpeed3pm     0.021050
Humidity9am      0.018246
Temp9am          0.012148
WindSpeed9am     0.012148
MinTemp          0.010209
MaxTemp          0.008669
Location         0.000000
year             0.000000
month            0.000000
day              0.000000
dtype: float64
Удалили признаки: {'Sunshine', 'Cloud3pm', 'Evaporation', 'Cloud9am'}


* Удалим объекты, где таргет RainTomorrow пропущен
* В категориальных признаках пропуски заполним наиболее частым значением
* В численных признаках пропуски заполним медианой

In [190]:
data.dropna(subset=['RainTomorrow'])
data.fillna(data.select_dtypes(include='object').mode().iloc[0], inplace=True)
data.fillna(data.select_dtypes(include='float64').median(), inplace=True)

С помощью IQR удалим выбросы в численных признаках

In [191]:
float_features = data.select_dtypes(include=np.float64).columns
for feature in float_features:
    q1 = data[feature].quantile(0.25)
    q3 = data[feature].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    data = data[(data[feature] >= lower) & (data[feature] <= upper)]

Применим LabelEncoder над категориальными признаками

In [192]:
categorical_columns = data.select_dtypes(include='object').columns
label_encoders = {column: LabelEncoder() for column in categorical_columns}
for column, encoder in label_encoders.items():
    data[column] = label_encoders[column].fit_transform(data[column])

In [193]:
data.head()

,Pressure9am,Pressure3pm,WindDir9am,WindGustDir,WindGustSpeed,Humidity3pm,WindDir3pm,Temp3pm,RainTomorrow,RainToday,...,WindSpeed3pm,Humidity9am,Temp9am,WindSpeed9am,MinTemp,MaxTemp,Location,year,month,day
0,1007.7,1007.1,13,13,44.0,22.0,14,21.8,0,0,...,24.0,71.0,16.9,20.0,13.4,22.9,2,2008,12,1
1,1010.6,1007.8,6,14,44.0,25.0,15,24.3,0,0,...,22.0,44.0,17.2,4.0,7.4,25.1,2,2008,12,2
2,1007.6,1008.7,13,15,46.0,30.0,15,23.2,0,0,...,26.0,38.0,21.0,19.0,12.9,25.7,2,2008,12,3
3,1017.6,1012.8,9,4,24.0,16.0,0,26.5,0,0,...,9.0,45.0,18.1,11.0,9.2,28.0,2,2008,12,4
4,1010.8,1006.0,1,13,41.0,33.0,7,29.7,0,0,...,20.0,82.0,17.8,7.0,17.5,32.3,2,2008,12,5


Обновим индексы для разбиений

In [194]:
indexes = {
    'train': (data['year'] >= 2007) & (data['year'] <= 2014),
    'val': data['year'] == 2015,
    'test': (data['year'] >= 2016) & (data['year'] <= 2017),
}

Ваш таргет - **RainTommorow**. Удалите его из обучающих данных, также удалите признак RISK_MM.

In [195]:
target_data = data['RainTomorrow']
data.drop(['RainTomorrow'], axis=1, inplace=True)

In [196]:
X_train, y_train = data[indexes['train']].values, target_data[indexes['train']].values
X_val, y_val = data[indexes['val']].values, target_data[indexes['val']].values
X_test, y_test = data[indexes['test']].values, target_data[indexes['test']].values

Для каждого из алгоритмов достигнутое качество (**accuracy**) должно быть: 
* ***RandomForest > 0.84 (2 points)***
* ***GradientBoosting > 0.845 (2 points)***
* ***Отчет об экспериментах (2 points)***

Обучите каждый из алгоритмов до нужного качества, используйте валидационную выборку, чтобы подбирать гиперпараметры. Получите качество (accuracy) выше необходимого и на validation, и на test.

**Подсказка:** для визуализации прогресса обучения можно использовать бибилиотеку [`tqdm`](https://tqdm.github.io/).

1. RandomForestClassifier

In [198]:
rf_params_grid = {
    'n_estimators': [30, 100],
    'max_objects_samples': [0.9, 1.0],
    'max_features_samples': [0.6, 0.8],
    'max_depth': [10, None],
    'min_samples_leaf': [1, 4],  
}

print('RandomForestClassifier')
best_rf_params = None
max_rf_acc = 0
for params in tqdm([dict(zip(rf_params_grid.keys(), v)) for v in itertools.product(*rf_params_grid.values())]):
    rf = RandomForestClassifier(random_state=42, **params)
    rf.fit(X_train, y_train)
    val_acc = accuracy_score(y_val, rf.predict(X_val))
    print(f"{val_acc=:.4f} {params}")
    if val_acc > max_rf_acc:
        best_rf_params = params
        max_rf_acc = val_acc        

print(f'Лучшие гиперпараметры RandomForestClassifier на валидации: {best_rf_params}')
print(f'Лучший accuracy RandomForestClassifier на валидации: {max_rf_acc:.3f}')

rf_best = RandomForestClassifier(random_state=42, **best_rf_params)
rf_best.fit(X_train, y_train)
rf_test_acc = accuracy_score(y_test, rf_best.predict(X_test))
print(f'Accuracy RandomForestClassifier на лучших подобранных гиперпараметрах: {rf_test_acc:.4f}')

RandomForestClassifier


  0%|          | 0/32 [00:00<?, ?it/s]

val_acc=0.8715 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.6, 'max_depth': 10, 'min_samples_leaf': 1}
val_acc=0.8710 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.6, 'max_depth': 10, 'min_samples_leaf': 4}
val_acc=0.8722 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.6, 'max_depth': None, 'min_samples_leaf': 1}
val_acc=0.8736 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.6, 'max_depth': None, 'min_samples_leaf': 4}
val_acc=0.8733 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.8, 'max_depth': 10, 'min_samples_leaf': 1}
val_acc=0.8734 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.8, 'max_depth': 10, 'min_samples_leaf': 4}
val_acc=0.8746 {'n_estimators': 30, 'max_objects_samples': 0.9, 'max_features_samples': 0.8, 'max_depth': None, 'min_samples_leaf': 1}
val_acc=0.8741 {'n_estimators': 30, 'max_objects_samples': 0.9,

2. GradientBoostingClassifier

In [199]:
gb_params_grid = {
    'n_estimators': [50, 150],
    'max_features_samples': [0.8, 1],
    'lr': [1e-3, 1e-2, 1e-1, 1],
    'max_depth': [1, 3, 5, None],
    'min_samples_leaf': [1, 4],
}

print('GradientBoostingClassifier')
best_gb_params = None
max_gb_acc = 0
for params in tqdm([dict(zip(gb_params_grid.keys(), v)) for v in itertools.product(*gb_params_grid.values())]):
    gb = GradientBoostingClassifier(random_state=42, **params)
    gb.fit(X_train, y_train)
    val_acc = accuracy_score(y_val, gb.predict(X_val))
    print(f"{val_acc=:.4f} {params}")
    if val_acc > max_gb_acc:
        best_gb_params = params
        max_gb_acc = val_acc        

print(f'Лучшие гиперпараметры GradientBoostingClassifier на валидации: {best_gb_params}')
print(f'Лучший accuracy GradientBoostingClassifier на валидации: {max_gb_acc:.3f}')

gb_best = GradientBoostingClassifier(random_state=42, **best_gb_params)
gb_best.fit(X_train, y_train)
gb_test_acc = accuracy_score(y_test, gb_best.predict(X_test))
print(f'Accuracy GradientBoostingClassifier на лучших подобранных гиперпараметрах: {gb_test_acc:.4f}')

GradientBoostingClassifier


  0%|          | 0/128 [00:00<?, ?it/s]

val_acc=0.8138 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 1, 'min_samples_leaf': 1}
val_acc=0.8138 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 1, 'min_samples_leaf': 4}
val_acc=0.8356 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 3, 'min_samples_leaf': 1}
val_acc=0.8356 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 3, 'min_samples_leaf': 4}
val_acc=0.8284 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 5, 'min_samples_leaf': 1}
val_acc=0.8257 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': 5, 'min_samples_leaf': 4}
val_acc=0.8035 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': None, 'min_samples_leaf': 1}
val_acc=0.8063 {'n_estimators': 50, 'max_features_samples': 0.8, 'lr': 0.001, 'max_depth': None, 'min_samples_leaf': 4}
val_acc=0.8138 {'n_estimators': 50, 'max_features_samples'

Выводы:
* Превысить бейзлайн на градиентном бустинге в результате помогли дополнительные предобработки (выбросы, отбрасывание признаков с высокой долей пропусков)
* RandomForestClassifier на предобработанном датасете превышает бейзлайн и на дефолтных параметрах. Лучший результат после перебора:
    * `'n_estimators': 100, 'max_objects_samples': 1.0, 'max_features_samples': 0.8, 'max_depth': None, 'min_samples_leaf': 4`
    * Accuracy на валидации: 0.879
    * Accuracy на тесте: 0.869
* GradientBoostingClassifier потребовал также большей сетки перебора для достижения бейзлайна. Лучший результат:
    * `'n_estimators': 150, 'max_features_samples': 1, 'lr': 0.001, 'max_depth': 1, 'min_samples_leaf': 1`. Получается, лучших метрик достигли с решающими пнями.
    * Accuracy на валидации: 0.866
    * Accuracy на тесте: 0.853

## 3. Бонус. [AdaBoost](https://ru.wikipedia.org/wiki/AdaBoost) (5 points)

В алгоритме AdaBoost всем объектам обучения присваивается вес `weight`, который определяет степень важности объекта при обучении. И если текущая модель ошибается на некотором объекте, его вес увеличивается, и этот объект будет больше влиять на обучение следующей модели. 

Также, так как модели обучаются последовательно, они не равносильны между собой, поэтому у каждой модели тоже есть вес `alpha`, который определяет вес модели при суммировании ответов и зависит от количества ошибок `err` модели. На каждой итерации обучения, эти веса пересчитываются по формулам:

$$\alpha_j = \log\left(\frac{1-err_j}{err_j}\right),$$
где $err_j$ - ошибка классификации

$$w_{new}^t = \frac{w_{old}^{t}*\exp(-\alpha_j \cdot y(x^t) \cdot b_j(x^t))}{\sum\limits_{i=1}^m w_{old}^{t}*\exp(-\alpha_j \cdot y(x^i) \cdot b_j(x^i))}$$

Изначально все веса объектов $w^i$ равны (и нормированы на 1).

Вам предлагается полностью реализовать AdaBoost. Вы можете использовать предыдущие шаблоны, но учтите некоторые пункты:
* надо работать с метками {-1,1} - это обусловлено использованием экспоненциальной функции потерь
* метод `predict` представляет собой функцию сигмоид, примененную к сумме предсказаний всех моделей

Для реализации AdaBoost на датасете, полученном в предыдущем пункте требуется добиться точноcти:
* AdaBoost > 0.83

Полезные ссылки:

- [статья](https://face-rec.org/algorithms/Boosting-Ensemble/decision-theoretic_generalization.pdf)
- [nerc](https://neerc.ifmo.ru/wiki/index.php?title=%D0%91%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3,_AdaBoost)
- [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)